In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features
from src.visualization import visualize

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

# Proyección de casos INT al 2035

En este cuadernillo se quiere estimar la deminada que atenderá el INT para el 2035. Con esta demanada se calcularán las cantidades de camas necesarias para suplir tal demanda. Para determinar la demanda se utilizarán:

1. Cantidad de casos teóricos por problemas de salud relevantes seleccionados: Estos se calcularon según la incidencia de cada problema de salud y la proyección de la población al 2035.

2. Cantidad de casos atendidos actualmente por cada problema de salud: Estos se obtuvieron al analizar la base de egresos hospitalarios DEIS.

Con ambos datos, se obtendrá cuanto % de atención ha cubierto el INT de los casos teóricos.

## Obtencion de casos teoricos para cada diagnóstico relevante segun incidencia

Estos seran obtenidos desde el cuadernillo 1.0-jrb-number-of-persons-per-strata

In [3]:
# Importa los casos teoricos por cada problema de salud
casos_teoricos = pd.read_excel("../data/interim/casos_teoricos_diagnosticos.xlsx")
casos_teoricos["Diagnostico"] = casos_teoricos["Diagnostico"].str.split("-").str[0].str.strip()

# Extrae los diagnosticos relevantes para el recinto hospitalario
DIAGNOSTICOS_RELEVANTES = casos_teoricos["Diagnostico"].unique()
casos_teoricos = casos_teoricos.set_index("Diagnostico")

# Renombra columnas
COLUMNAS_A_RENOMBRAR = {
    "Poblacion 2017": 2017,
    "Poblacion 2018": 2018,
    "Poblacion 2019": 2019,
    "Poblacion 2020": 2020,
    "Poblacion 2021": 2021,
    "Poblacion 2022": 2022,
    "Poblacion 2023": 2023,
    "Poblacion 2024": 2024,
    "Poblacion 2025": 2025,
    "Poblacion 2026": 2026,
    "Poblacion 2027": 2027,
    "Poblacion 2028": 2028,
    "Poblacion 2029": 2029,
    "Poblacion 2030": 2030,
    "Poblacion 2031": 2031,
    "Poblacion 2032": 2032,
    "Poblacion 2033": 2033,
    "Poblacion 2034": 2034,
    "Poblacion 2035": 2035,
}
casos_teoricos = casos_teoricos.rename(columns=COLUMNAS_A_RENOMBRAR)

# Obtiene los casos en el estrato nacional
casos_nacionales = casos_teoricos.query("Estrato == 'Pais' and Estadística == 'Incidencia'").copy()
casos_nacionales_del_2017_al_2020 = casos_nacionales[[i for i in range(2017, 2021)]]

In [12]:
print(
    f"Los diagnosticos que seran proyectados son: \n{DIAGNOSTICOS_RELEVANTES} "
    f"\ny son {len(DIAGNOSTICOS_RELEVANTES)}"
)

Los diagnosticos que seran proyectados son: 
['A152' 'B441' 'C341' 'C342' 'C343' 'C381' 'C780' 'C782' 'D381' 'E848'
 'I051' 'I052' 'I080' 'I081' 'I232' 'I330' 'I340' 'I351' 'I352' 'I361'
 'I420' 'I421' 'I495' 'I710' 'I711' 'I712' 'I715' 'J679' 'J841' 'J848'
 'J955' 'M051' 'M348' 'Q222' 'Q233' 'Q257' 'Z450' 'I350'] 
y son 38


## Obtención de casos atendidos entre 2017 y 2020 para diagnósticos relevantes.

Estos se obtendrán del análisis de egresos hospitalarios a nivel país DEIS.

In [4]:
# Carga los egresos nacionales
egresos_nacionales = pd.read_csv(
    "../data/processed/ranking_nacional_egresos.csv", sep=";", encoding="latin-1"
)

# Filtra solamente los datos del INT
egresos_torax = egresos_nacionales.query("ESTABLECIMIENTO_SALUD == 112103").copy()

En primer lugar, se filtrarán ambas bases de datos para solamente tener la información de los diagnósticos más relevantes para el INT. Luego de esto, se calculará la cantidad de egresos, dias de estada y pacientes para cada uno de los diagnósticos. Estos insumos serán utilizados para estimar la demanda.


In [13]:
# Filtra solamente los diagnosticos mas relevantes del Torax
egresos_mas_relevantes_nacionales = (
    egresos_nacionales[egresos_nacionales["DIAG1"].isin(DIAGNOSTICOS_RELEVANTES)]
    .query("ANO_EGRESO >= 2017")
    .copy()
)

# Filtra solamente los diagnosticos relevantes para el torax
egresos_mas_relevantes_torax = egresos_mas_relevantes_nacionales.query(
    "ESTABLECIMIENTO_SALUD == 112103"
).copy()

In [31]:
# Obtiene el resumen de metricas para el estrato nacional
metricas_diags_relevantes_nacionales = build_features.calculate_discharges_metrics(
    egresos_mas_relevantes_nacionales
)

# Obtiene resumen de metricas para el Torax
metricas_diags_relevantes_torax = build_features.calculate_discharges_metrics(
    egresos_mas_relevantes_torax
)

# Obtiene el porcentaje de resolucion de egresos del Torax
porcentaje_egresos_nacionales_torax = (
    metricas_diags_relevantes_torax["n_egresos"] / metricas_diags_relevantes_nacionales["n_egresos"]
)

porcentaje_egresos_nacionales_torax_acumulado = (
    metricas_diags_relevantes_torax["n_egresos"].sum(axis=1)
    / metricas_diags_relevantes_nacionales["n_egresos"].sum(axis=1)
).to_frame()

In [33]:
# Muestra el porcentaje de atencion del INT para ciertos problemas de salud
print("Porcentaje de atencion (egresos) para ciertos problemas de salud relevantes INT")
display(porcentaje_egresos_nacionales_torax.head(10).style.format("{:,.2%}"))

print("Porcentaje de atencion (egresos) acumulada para ciertos problemas de salud relevantes INT")
display(porcentaje_egresos_nacionales_torax_acumulado.head(10).style.format("{:,.2%}"))

Porcentaje de atencion (egresos) para ciertos problemas de salud relevantes INT


,2017,2018,2019,2020
DIAG1,,,,
A152,31.58%,90.68%,46.15%,0.00%
B441,25.00%,9.09%,31.03%,44.44%
C341,48.39%,46.03%,44.62%,24.05%
C342,38.00%,35.29%,38.46%,14.58%
C343,45.74%,42.32%,43.01%,17.04%
C381,36.59%,51.22%,36.54%,11.90%
C780,15.28%,16.62%,14.42%,4.60%
C782,19.73%,28.19%,18.48%,25.50%
D381,6.51%,13.26%,14.83%,8.06%


Porcentaje de atencion (egresos) acumulada para ciertos problemas de salud relevantes INT


,0
DIAG1,
A152,70.62%
B441,25.00%
C341,41.95%
C342,32.11%
C343,37.99%
C381,34.09%
C780,13.06%
C782,23.06%
D381,10.91%


Solamente se compararán los casos reales vs teóricos entre 2017 y 2020, ya que la información se encuentra disponible en tales periodos.

En primer lugar, se obtendrá el porcentaje de atención del INT del total de casos teóricos. O sea:

$$\%atencion = \frac{Casos Reales}{Casos Teoricos}$$

La cantidad de casos están desglosados por año, por lo que es posible calcular un porcentaje de atención anual. Sin embargo, para efectos de cálculo, se obtendrá el porcentaje acumulado entre los años disponibles (2017 a 2020).

$$\%Atencion\ Acumulada = \frac{\sum_{2017}^{2020}Casos Reales}{\sum_{2017}^{2020}Casos Teoricos}$$

In [38]:
# Obtiene cuanto se hace cargo el INT de los pacientes teoricos
porcentaje_atencion_teorica_int = (
    metricas_diags_relevantes_torax["n_pacientes_distintos"].sum(axis=1)
    .div(casos_nacionales_del_2017_al_2020.sum(axis=1), axis=0)
)

In [40]:
display(porcentaje_atencion_teorica_int.to_frame().head(10).style.format("{:,.2%}"))

,0
A152,0.14%
B441,0.11%
C341,1.83%
C342,0.20%
C343,1.06%
C381,5.94%
C780,0.57%
C782,0.52%
D381,0.73%
E848,76.96%


Una vez se sabe la proporcion de los casos que atiende el INT históricamente, se procederá a estimar cuántos casos atenderá si es que se mantiene en las mismas condiciones actuales (mismo porcentaje de atención del nacional, mismos egresos por personas y mismos dias de estada por persona).

In [45]:
# Se obtiene la cantidad de casos que debiese atender el Torax, asumiendo que atendera
# lo mismo que entre 2017 y 2020 del total de casos teoricos
cantidad_de_casos_estimados_int = casos_nacionales[[i for i in range(2017, 2036)]].mul(
    porcentaje_atencion_teorica_int, axis=0
)

# Se obtiene la cantidad de egresos que debiese tener el INT, asumiendo que trabajara con la misma
# eficiencia
egresos_estimados_int = cantidad_de_casos_estimados_int.mul(
    metricas_diags_relevantes_torax["egresos_por_paciente_agrupado"]["2017-2020"], axis=0
)

# Se obtiene la cantidad de dias de estada que debiese el INT, asumiendo que trabajara con la misma
# eficiencia
dias_estada_estimados_int = egresos_estimados_int.mul(
    metricas_diags_relevantes_torax["dias_estada_promedio_agrupado"]["2017-2020"], axis=0
)

# Estima la cantidad de camas necesaarias por diagnostico
camas_estimadas_int = (dias_estada_estimados_int / 365.25) * 1.2

In [46]:
camas_estimadas_int.sum()

2017    38.511028
2018    39.292806
2019    40.165832
2020    41.057850
2021    41.716284
2022    42.240337
2023    42.729819
2024    43.206157
2025    43.673491
2026    44.133081
2027    44.586747
2028    45.034088
2029    45.472794
2030    45.899854
2031    46.329011
2032    46.737233
2033    47.132964
2034    47.514748
2035    47.879825
dtype: float64